# SEIRX model runs for test technology and screening frequency investigation

In [1]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

In [2]:
# paths for I/O
contact_network_src = '../data/contact_networks'

## Simulation function

In [3]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''

    # extract the simulation parameters from the parameter list
    index_case, e_vaccination_ratio, r_vaccination_ratio, N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params,
                contact_network_src, ensmbl_dst, index_case=index_case, 
                e_vaccination_ratio=e_vaccination_ratio,
                r_vaccination_ratio=r_vaccination_ratio)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case,
                                e_vaccination_ratio=e_vaccination_ratio,
                                r_vaccination_ratio=r_vaccination_ratio)

    return row

## Screening parameters

In [4]:
N_runs = 5000

e_vaccination_ratios = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
r_vaccination_ratios = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

screening_params = [(i, j, k, N_runs)\
              for i in index_cases \
              for j in e_vaccination_ratios \
              for k in r_vaccination_ratios]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 242 parameter combinations


## B.1.1.7

### Simulation parameters and measures in place

In [5]:
with open('params/vaccination_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/vaccination_measures.json') as json_file:
    measures = json.load(json_file)


In [6]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.109549,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [7]:
measures

{'testing': 'diagnostic',
 'preventive_screening_test_type': None,
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': False,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'transmission_risk_vaccination_modifier': {'reception': 0.6,
  'transmission': 0.3},
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [8]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/vaccination/UK_variant'

number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_vaccination_rate_UK_variant_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)

100%|██████████| 242/242 [3:24:09<00:00, 50.62s/it]   


CPU times: user 5.58 s, sys: 656 ms, total: 6.24 s
Wall time: 3h 24min 13s


,index_case,resident_vaccination_ratio,employee_vaccination_ratio,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,employee,1.0,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,2.006684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,0.9,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,2.699763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,employee,0.8,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,3.632585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Wild type

In [9]:
ensmbl_dst = '../data/simulation_results/ensembles/vaccination/wild_type'

### Simulation parameters and measures in place

In [10]:
with open('params/vaccination_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/vaccination_measures.json') as json_file:
    measures = json.load(json_file)


In [11]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.073033,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [12]:
measures

{'testing': 'diagnostic',
 'preventive_screening_test_type': None,
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': False,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'transmission_risk_vaccination_modifier': {'reception': 0.6,
  'transmission': 0.3},
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [13]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 4 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_vaccination_rate_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)

100%|██████████| 242/242 [2:42:48<00:00, 40.36s/it]   


CPU times: user 5.43 s, sys: 414 ms, total: 5.85 s
Wall time: 2h 42min 52s


,index_case,resident_vaccination_ratio,employee_vaccination_ratio,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,employee,1.0,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.250011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,0.9,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.492430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,employee,0.8,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.997937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## B.1.1.7 & masks for employees

### Simulation parameters and measures in place

In [14]:
with open('params/vaccination_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/vaccination_masks_measures.json') as json_file:
    measures = json.load(json_file)


In [15]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.109549,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [16]:
measures

{'testing': 'diagnostic',
 'preventive_screening_test_type': None,
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': True,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'transmission_risk_vaccination_modifier': {'reception': 0.6,
  'transmission': 0.3},
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [17]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/vaccination/masks'

number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_vaccination_rate_masks_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)

100%|██████████| 242/242 [2:42:04<00:00, 40.18s/it]   


CPU times: user 5.57 s, sys: 429 ms, total: 6 s
Wall time: 2h 42min 8s


,index_case,resident_vaccination_ratio,employee_vaccination_ratio,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,N_agents_mean,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,employee,1.0,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,0.673072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,0.9,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,0.872880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,employee,0.8,0.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.266857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
